In [1]:
  # Real time network
# --- Install required packages (run this cell once) ---
import sys
import subprocess

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

packages = ["psutil", "plotly", "pandas", "ipywidgets", "anywidget"]
for p in packages:
    try:
        __import__(p)
    except Exception:
        print(f"Installing {p}...")
        install(p)

# enable widgets for classic notebook
try:
    from notebook.nbextensions import install_nbextension_python, enable_nbextension_python
    # enable widgetsnbextension
    subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"])
except Exception:
    # best effort; may already be enabled
    pass

print("✅ Install step finished. Restart the kernel if required, then run the main cell.")


✅ Install step finished. Restart the kernel if required, then run the main cell.


In [ ]:

# --- Main real-time network line chart cell ---
import psutil
import pandas as pd
import plotly.graph_objects as go
import time
from IPython.display import display, clear_output
import ipywidgets as widgets
from collections import deque

# UI: Stop button
run_flag = {"run": True}
def stop_execution(b):
    run_flag["run"] = False

stop_button = widgets.Button(description="🛑 Stop Live Chart", button_style="danger")
stop_button.on_click(stop_execution)
display(stop_button)

# Controls: refresh interval and max points
refresh_box = widgets.BoundedIntText(value=1, min=1, max=10, description='Refresh (s):')
max_points_box = widgets.BoundedIntText(value=60, min=10, max=1000, description='Max points:')
display(widgets.HBox([refresh_box, max_points_box]))

# Data containers
times = deque(maxlen=max_points_box.value)
uploads = deque(maxlen=max_points_box.value)
downloads = deque(maxlen=max_points_box.value)

# initial counters
net0 = psutil.net_io_counters()
old_sent = net0.bytes_sent
old_recv = net0.bytes_recv

# Create plotly FigureWidget
fig = go.FigureWidget()
fig.add_scatter(name='Upload (KB/s)', mode='lines', line=dict(color='firebrick'))
fig.add_scatter(name='Download (KB/s)', mode='lines', line=dict(color='royalblue'))
fig.update_layout(title='📈 Real-Time Network Upload & Download (KB/s)',
                  xaxis_title='Time (s)',
                  yaxis_title='Speed (KB/s)',
                  template='plotly_dark')
display(fig)

start = time.time()
# Main loop
while run_flag["run"]:
    try:
        refresh = int(refresh_box.value)
        maxp = int(max_points_box.value)
        # update deque sizes if user changed max points
        times = deque(times, maxlen=maxp)
        uploads = deque(uploads, maxlen=maxp)
        downloads = deque(downloads, maxlen=maxp)

        net = psutil.net_io_counters()
        new_sent = net.bytes_sent
        new_recv = net.bytes_recv

        upload_kbps = (new_sent - old_sent) / 1024.0 / refresh
        download_kbps = (new_recv - old_recv) / 1024.0 / refresh
        old_sent, old_recv = new_sent, new_recv

        current_time = round(time.time() - start, 1)
        times.append(current_time)
        uploads.append(round(upload_kbps, 2))
        downloads.append(round(download_kbps, 2))

        with fig.batch_update():
            fig.data[0].x = list(times)
            fig.data[0].y = list(uploads)
            fig.data[1].x = list(times)
            fig.data[1].y = list(downloads)
            # adjust xaxis range to show last points
            if len(times) > 1:
                fig.update_xaxes(range=[times[0], times[-1]])

        time.sleep(refresh)
    except Exception as e:
        clear_output(wait=True)
        print('Stopped due to error:', e)
        break

clear_output(wait=True)
print('✅ Live chart stopped safely.')


Button(button_style='danger', description='🛑 Stop Live Chart', style=ButtonStyle())

FigureWidget({
    'data': [{'line': {'color': 'firebrick'},
              'mode': 'lines',
              'name': 'Upload (KB/s)',
              'type': 'scatter',
              'uid': '4eb90197-228f-404d-a3a0-9ef58c753d11'},
             {'line': {'color': 'royalblue'},
              'mode': 'lines',
              'name': 'Download (KB/s)',
              'type': 'scatter',
              'uid': 'e6d40ca4-854b-48f0-8155-7629129066e2'}],
    'layout': {'template': '...',
               'title': {'text': '📈 Real-Time Network Upload & Download (KB/s)'},
               'xaxis': {'title': {'text': 'Time (s)'}},
               'yaxis': {'title': {'text': 'Speed (KB/s)'}}}
})